In [ ]:
import os
import datetime
from pathlib import Path
from collections import defaultdict
import scipy
import random
import numpy as np
import xarray as xr
import pandas as pd
import joblib
import pickle
import xesmf as xe
import glob

import seaborn as sns
import cmocean as cm            # really nice colorbars
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy import stats
import gcsfs 

import requests
import io

In [ ]:
# Import data file from google cloud
fs = gcsfs.GCSFileSystem()  # initializes cloud storage interface

input_data_path = 'YOUR_GCS_PATH/Taylor_data/databases/processed'


In [ ]:
fs.ls(input_data_path)

In [ ]:
fs.ls('YOUR_GCS_PATH/Taylor_data')

In [ ]:
BATS_path = input_data_path + '/bats_spco2_1988-10-2023-06.zarr'
HOT_path = input_data_path + '/HOT_spco2_202312.zarr'
LDEO_path = input_data_path + '/LDEOv2019_spco2.zarr'
GLODAP_path = input_data_path + '/GLODAPv2_spco2_2023.zarr'

# SOCCOM_path = 

In [ ]:
def BATS_stats(ml_timeseries,recon,start_yr=1991,end_yr=2023,blat=121,blon=115):
    
    # determine appropriate START YEAR # 
    start_yr = max(start_yr, 1991)
    end_yr = min(end_yr, 2023)
    
    # Grab BATS observations:
    bats = xr.open_dataset(BATS_path, engine='zarr')
    
    # Remove duplicates
    bats = bats.groupby('time').mean()
    # Put BATS monthly averages:
    bats_monthly = bats.spco2.to_dataframe().resample('ME').mean()
    # Extract years of interest:
    bats_monthly = bats_monthly.loc[f'{start_yr}-1-06 00:00:00':f'{end_yr}-12-31 23:59:59']
    # 31 50'N 64 10'W ###########     
    
    # Model already monthly #
    pco2_df = ml_timeseries[:,blat,blon].squeeze().to_dataframe()
    if start_yr == 1991:
        pco2_df = pco2_df.loc[f'{start_yr}-10-01 00:00:00':f'{end_yr}-12-31 23:59:59'] 
    else:
        pco2_df = pco2_df.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 23:59:59']    
    
    # get time uniform:
    bats_monthly = bats_monthly.set_index(pco2_df.index)
    
    # correlate where no NANs
    fib = (~np.isnan(bats_monthly.spco2) & ~np.isnan(pco2_df[recon]))
    R = np.corrcoef(pco2_df[recon][fib],bats_monthly.spco2[fib])[0,1]
    STD = np.nanstd(pco2_df[recon][fib])
    BATS_STD = np.nanstd(bats_monthly.spco2[fib])
    
    #RMSE
    RMSE = np.sqrt(np.square(bats_monthly.spco2[fib]-pco2_df[recon][fib]).sum()/(fib.sum()))
    
    return R, STD, RMSE, BATS_STD

In [ ]:
def HOT_stats(ml_timeseries,recon,start_yr=1988,end_yr=2022,hlat=112,hlon=22):
    
    # determine appropriate START YEAR # 
    start_yr = max(start_yr, 1988)
    end_yr = min(end_yr, 2022)
    
    # Grab observations
    hot = xr.open_mfdataset(HOT_path, engine= 'zarr') 
    hot = hot.where(hot.spco2>0)
    
    pco2_df = ml_timeseries[:,hlat,hlon].squeeze().to_dataframe()
    if start_yr == 1988:
        pco2_df = pco2_df.loc[f'{start_yr}-10-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    else:
        pco2_df = pco2_df.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
        
    
    # P
    if not isinstance(hot.spco2.to_dataframe().index, pd.DatetimeIndex):
        df_temp = hot.spco2.to_dataframe()
        df_temp.index = pd.to_datetime(df_temp.index)
        hots_monthly = df_temp.resample('ME').mean()  
    else:
        hots_monthly = hot.spco2.to_dataframe().resample('ME').mean()
    hots_monthly = hots_monthly.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    hots_monthly = hots_monthly.set_index(pco2_df.index)
    # Point by point comparison #
    ind = ((~np.isnan(hots_monthly.spco2)) & (hots_monthly.spco2 > 150) & (~np.isnan(pco2_df[recon])))
    R = np.corrcoef(pco2_df[recon][ind],hots_monthly.spco2[ind])[0,1]
    
    STD = np.std(pco2_df[recon][ind])
    HOT_STD = np.nanstd(hots_monthly.spco2[ind])
    
    #RMSE
    RMSE = np.sqrt(np.square(hots_monthly.spco2[ind]-pco2_df[recon][ind]).sum()/ind.sum())
    
    # We want to know the correlation coef, STD of reconstruction, trendline, seasonal cycle:
    return R, STD, RMSE, HOT_STD

In [ ]:
def LDEO_stats(ml_gridded,recon,start_yr=1957,end_yr=2019):
    
    # determine appropriate START YEAR # 
    start_yr = max(start_yr, 1957)
    end_yr = min(end_yr, 2019)

    # if recon in ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','spco2','HPD','resid']:
    # lon = ml_gridded.lon
    # lat = ml_gridded.lat
    # else:
        #     lon = ml_gridded.xlon
    #     lat = ml_gridded.ylat
    if 'lon' in ml_gridded.coords:
        lon = ml_gridded.lon
    else:
        lon = ml_gridded.xlon
    
    if 'lat' in ml_gridded.coords:
        lat = ml_gridded.lat
    else:
        lat = ml_gridded.ylat
        
    mtime = ml_gridded.time

    # Load the observations
    ldeo = xr.open_mfdataset(LDEO_path, engine= 'zarr') 
    ldeo_pco2=ldeo.spco2_mean[(start_yr-1985)*12:(end_yr)*12,:,:]
    ldeo_lat=ldeo.lat
    ldeo_lon=ldeo.lon
    ldeo_time = ldeo.time
    
    # Regrid to LDEO's grid #########
    mgrid = xr.Dataset({'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    lgrid = xr.Dataset({'lat':(['lat'],ldeo_lat.values),'lon':(['lon'],ldeo_lon.values)})
    mpco2 = xr.Dataset({recon:(['time','lat','lon'],ml_gridded.values),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    regridder = xe.Regridder(mpco2, lgrid, 'bilinear',periodic = True)
    pco2_new = regridder(ml_gridded)

    ldeo_pco2 = ldeo_pco2.where(((ldeo_pco2<850) & (ldeo_pco2>150)))
    ldeo_pco2 = ldeo_pco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    pco2_new = pco2_new.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    

    # stack
    lpco2 = ldeo_pco2.stack(level=['time','lat','lon'])
    pco2_stack = pco2_new.stack(level=['time','lat','lon'])
    
    indx = ((~np.isnan(lpco2.values)) & (~np.isnan(pco2_stack.values)) & (pco2_stack.values > 50))
    # print("LDEO OBS # =",sum(indx))

    R = np.corrcoef(lpco2[indx],pco2_stack[indx])[0,1]
    LDEO_STD = np.std(lpco2[indx].values)
    STD = np.std(pco2_stack[indx].values)
    #RMSE
    RMSE = np.sqrt(np.square(lpco2[indx].values-pco2_stack[indx].values).sum()/indx.sum())
    
    return R, STD, RMSE, LDEO_STD

In [ ]:
def GLODAP_stats(ml_gridded,recon,start_yr=1986,end_yr=2021):
    
    # Reconstructions #
    # if recon in ['CSIR_ML6','JENA_MLS','JMA_MLR','MPI_SOMFFN','NIES_FNN','CMEMS_FFNN','spco2','HPD','resid']:
    # lon = ml_gridded.lon
    # lat = ml_gridded.lat
    # else:
    #     lon = ml_gridded.xlon
    #     lat = ml_gridded.ylat

    if 'lon' in ml_gridded.coords:
        lon = ml_gridded.lon
    else:
        lon = ml_gridded.xlon
    
    if 'lat' in ml_gridded.coords:
        lat = ml_gridded.lat
    else:
        lat = ml_gridded.ylat
        
    start_yr = max(start_yr,1986)
    end_yr = min(end_yr,2021)
    
    mtime = ml_gridded.time
    
    # Load the data #
    glod = xr.open_mfdataset(GLODAP_path, engine= 'zarr') 
    glod_pco2=glod.spco2_mean
    glod_lat=glod.lat
    glod_lon=glod.lon
    glod_time = glod.time
    
    # Deal with NaNs #
    tmp = ml_gridded.where(ml_gridded > 0)
    tmp = tmp.where(tmp < 850)
    
    # Regrid to GLODAP grid #
    mgrid = xr.Dataset({'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    ggrid = xr.Dataset({'lat':(['lat'],glod_lat.values),'lon':(['lon'],glod_lon.values)})
    # mpco2 = xr.Dataset({recon:(['time','lat','lon'、],tmp),'time':(['time'],mtime.values),'lat':(['lat'],lat.values),'lon':(['lon'],lon.values)})
    mpco2 = xr.Dataset({
    recon: (['time', 'lat', 'lon'], tmp.data), 
    'time': (['time'], mtime.values),
    'lat': (['lat'], lat.values),
    'lon': (['lon'], lon.values)
    })
    
    regridder = xe.Regridder(mpco2, ggrid, 'bilinear',periodic = True) #,'periodic'
    pco2_new = regridder(tmp)
    
    # Extract Time #
    glod_pco2 = glod_pco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    glod_pco2 = glod_pco2.where(((glod_pco2 >200) & (glod_pco2 < 600)))
    if start_yr == 1986:
        pco2_new = pco2_new.loc[f'{start_yr}-7-15 00:00:00':f'{end_yr}-12-31 00:00:00']
    else:
        pco2_new = pco2_new.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    
    # Stack
    gpco2_stack = glod_pco2.stack(level=['time','lat','lon'])
    pco2_stack = pco2_new.stack(level=['time','lat','lon'])
    
    # Stats
    indx = ((~np.isnan(gpco2_stack.values)) & (~np.isnan(pco2_stack.values)) & (pco2_stack.values>50))
    R = np.corrcoef(gpco2_stack[indx],pco2_stack[indx])[0,1]
    GLODAP_STD = np.std(gpco2_stack[indx].values)
    STD = np.std(pco2_stack[indx].values)
    #RMSE
    RMSE = np.sqrt(np.square(gpco2_stack[indx].values-pco2_stack[indx].values).sum()/indx.sum())
    
    return R, STD, RMSE, GLODAP_STD

In [ ]:
# Set up dataframe #
####################
df = pd.DataFrame() # create empty data frame
df['stats']= ["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]
df.set_index([pd.Index(["R_BATS","STD_BATS","RMSE_BATS","R_HOT","STD_HOT","RMSE_HOT", "R_LDEO","STD_LDEO","RMSE_LDEO", "R_GLODAP","STD_GLODAP","RMSE_GLODAP"]), 'stats'])


In [ ]:
# Set up the time rage we want
start_yr = 1990 
end_yr = 2020

Residual

In [ ]:
# update the Residual product. download from zenodo.
# get the url from https://zenodo.org/records/7636890
# resid_url = "https://zenodo.org/records/7636890/files/LDEO-resid_1985-2021.nc?download=1"

### If you want to download from zenodo and not to save it on the GCS, use these code.
# response = requests.get(resid_url)
# response.raise_for_status()  # make sure download successfully
# # Save to temporary memory
# data_in_memory = io.BytesIO(response.content)
# # open as xarray
# ds = xr.open_dataset(data_in_memory, engine='h5netcdf', chunks={})
# gridded = ds.pco2
# # resid_product = ds.pco2
# recon = 'pco2' #just what the variable is called.
####

# Here we use the 2022 version from LEAP Pangeo
resid_url = 'gs://leap-persistent/galenmckinley/reconstructions/pCO2_LEAP_XGBoost-fco2-residual-reconstructed_198201-202212.zarr'
# open as xarray
ds = xr.open_mfdataset(resid_url,engine= 'zarr')

gridded = ds.fco2_reconstructed
recon = 'fco2_reconstructed' #just what the variable is called.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, b_obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, h_obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, s_obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, l_obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, g_obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['Residual']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
#put the observered on there too.
df['observed']=[1,b_obs,0,1,h_obs,0,1,l_obs,0,1,g_obs,0] #not sure exactly but doing what val does with this

In [ ]:
df.head()

HPD product 

In [ ]:
# update and grab HPD product. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
HPD_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_LDEO-HPD_1959-2022.nc?download=1"
directory = 'YOUR_GCS_PATH/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_LDEO-HPD_1959-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(HPD_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

In [ ]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2' #just what the variable is called.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['HPD']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

CMEMS-LSCE-FFNN

In [ ]:
# CMEMS. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
CMEMS_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_CMEMS-LSCE-FFNN_1990-2022.nc?download=1"
directory = 'YOUR_GCS_PATH/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_CMEMS-LSCE-FFNN_1990-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(CMEMS_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

In [ ]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

#CMEMS is too short (starts in 1990) so need to add blank months before 1990 to make it the same size?
gridded2 = gridded.loc['1992-1-01 00:00:00':'1999-12-31 00:00:00']
gridded2 = gridded2.where(gridded2 < 0) #this turns anything that is 0 or greater (not true in this equation) to nan.

earlytime = pd.date_range(start='1982-01-01', end='1989-12-31',freq='MS') + np.timedelta64(14, 'D') 

gridded2 = gridded2.assign_coords(time=earlytime) #overwrite time dimension to be midmonth

gridded2.coords['lon'] = (gridded2.coords['lon'] + 180) % 360 - 180
gridded2 = gridded2.sortby(gridded2.lon)

#concat gridded2 and gridded for CMEMS:
gridded_long = xr.concat([gridded2,gridded],dim='time')

recon = 'sfco2' #need this to work with *****_stats functions above. But not sure what to make it or how to not need it.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['CMEMS']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

JENA-MLS

In [ ]:
# JENA. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
JENA_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_JENA-MLS_1957-2022.nc?download=1"
directory = 'YOUR_GCS_PATH/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_JENA-MLS_1957-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(JENA_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

In [ ]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)
recon = 'sfco2'  #just what the variable is called.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['JENA']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

JMA-MLR

In [ ]:
# JMA. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
JMA_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_JMA-MLR_1985-2022.nc?download=1"
directory = 'YOUR_GCS_PATH/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_JMA-MLR_1985-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(JMA_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

In [ ]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['JMA']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

NIES-ML3

In [ ]:
# NIES. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
NIES_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_NIES-ML3_1980-2022.nc?download=1"
directory = 'YOUR_GCS_PATH/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_NIES-ML3_1980-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(NIES_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

In [ ]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00'] 

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['NIES']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

SOM-FFN

In [ ]:
# SOMFFN. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
SOMFFN_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_SOM-FFN_1982-2022.nc?download=1"
directory = 'YOUR_GCS_PATH/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_SOM-FFN_1982-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(SOMFFN_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

In [ ]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00'] 
#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['SOMFFN']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

UoEX_WATv2

In [ ]:
# UoEX_WAT. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
WAT_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_UoEX_WATv2_1985-2022.nc?download=1"
directory = 'YOUR_GCS_PATH/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_UoEX_WATv2_1985-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(WAT_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

In [ ]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['UoEX_WAT']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

OceanSODA-ETHZ

In [ ]:
# OceanSODA-ETHZ. download from zenodo.
# get the url from: https://zenodo.org/records/10222484
OceanSODA_url = "https://zenodo.org/records/10222484/files/GCB-2023_dataprod_OceanSODA-ETHZ_1982-2022.nc?download=1"
directory = 'YOUR_GCS_PATH/Taylor_data/data_products'
# Edit the file name if needed
save_path = f'{directory}/GCB-2023_dataprod_OceanSODA-ETHZ_1982-2022.zarr'

fs = gcsfs.GCSFileSystem()

# Check if we already have the file on GCS
if fs.exists(save_path):
    print(f"File already exists at {save_path}. Loading from GCS.")
    ds = xr.open_mfdataset(save_path, engine = 'zarr')
else:
    print(f"File does not exist at {save_path}. Downloading and saving.")
    response = requests.get(OceanSODA_url)
    if response.status_code == 200:
        with open('temp.nc', 'wb') as temp_file:
            temp_file.write(response.content)
        ds = xr.open_dataset('temp.nc')
        ds.to_zarr(save_path, mode='w')
        print(f"Successfully downloaded and saved to {save_path}")
        os.remove('temp.nc')
    else:
        print(f"Error! Could not download the file: {response.status_code}")

In [ ]:
# gridded = ds.sfco2
gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']

#regrid longitude
gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
gridded = gridded.sortby(gridded.lon)

recon = 'sfco2'  #just what the variable is called.

In [ ]:
R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr) # handle starts/ends in routines

# R_SOCCOM, STD_SOCCOM, RMSE_SOCCOM, obs = SOCCOM_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

In [ ]:
df['OceanSODA']=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]
df

Models

In [ ]:
# list of all models and their download url
# edit if necessary
models = ['ACCESS', 'CESM_ETHZ', 'CNRM_ESM2', 'FESOM2_REcoM', 'IPSL', 'MOM6_Princeton', 'MPIOM_HAMOCC', 'MRI_ESM2_2', 
          'NEMO_PlankTOM12', 'NorESM_OC1_2']
urls = ['https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_ACCESS_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_CESM_ETHZ_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_CNRM_ESM2_1_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_FESOM2_REcoM_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_IPSL_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_MOM6_Princeton_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_MPIOM_HAMOCC_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_MRI_ESM2_2_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_NEMO_PlankTOM12_1959-2022.nc?download=1',
        'https://zenodo.org/records/10222484/files/GCB-2023_OceanModel_NorESM_OC1_2_1959-2022.nc?download=1'
       ]

In [ ]:
directory = 'YOUR_GCS_PATH/Taylor_data/GOBMs'
for model, url in zip(models, urls):
    file_name = url.split('/')[-1].split('.nc')[0]
    save_path = f'{directory}/{file_name}.zarr'
    
    fs = gcsfs.GCSFileSystem()
    
    # Check if we already have the file on GCS
    if fs.exists(save_path):
        print(f"File already exists at {save_path}. Loading from GCS.")
        ds = xr.open_mfdataset(save_path, engine = 'zarr')
    else:
        print(f"File does not exist at {save_path}. Downloading and saving.")
        response = requests.get(url)
        if response.status_code == 200:
            with open('temp.nc', 'wb') as temp_file:
                temp_file.write(response.content)
            ds = xr.open_dataset('temp.nc')
            ds.to_zarr(save_path, mode='w')
            print(f"Successfully downloaded and saved to {save_path}")
            os.remove('temp.nc')
        else:
            print(f"Error! Could not download the file: {response.status_code}")
    
    # gridded = ds.sfco2
    # gridded = ds.sfco2.loc[f'{start_yr}-1-01 00:00:00':f'{end_yr}-12-31 00:00:00']
    gridded = ds.sfco2.sel(simulation='A').sel(time=slice(f'{start_yr}-01-01T00:00:00', f'{end_yr}-12-31T23:59:59'))
    
    #regrid longitude
    gridded.coords['lon'] = (gridded.coords['lon'] + 180) % 360 - 180
    gridded = gridded.sortby(gridded.lon)
    
    recon = 'sfco2'  #just what the variable is called.

    R_BATS, STD_BATS, RMSE_BATS, obs = BATS_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)
    R_HOT, STD_HOT, RMSE_HOT, obs = HOT_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)
    R_LDEO, STD_LDEO, RMSE_LDEO, obs = LDEO_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)
    R_GLODAP, STD_GLODAP, RMSE_GLODAP, obs = GLODAP_stats(gridded,recon,start_yr=start_yr,end_yr=end_yr)

    df[model]=[R_BATS,STD_BATS,RMSE_BATS,R_HOT,STD_HOT,RMSE_HOT, R_LDEO,STD_LDEO,RMSE_LDEO, R_GLODAP,STD_GLODAP,RMSE_GLODAP]

df

In [ ]:
# to show the statistics of Models
df.iloc[:,11:]

Save to GCS

In [ ]:
# Write out Files #

# Get current time
now = datetime.datetime.now()
current_year = now.year
current_month = str(now.month).zfill(2)
current_day = str(now.day).zfill(2)

output_path = 'YOUR_GCS_PATH/Taylor_data/Taylor_inputs'

# Edit the file name here
# if we set the time range from earlier than 1986 to later than 2022, then we assume it outputs the full time rage version
if (start_yr <= 1986 & end_yr >= 2022) or (start_yr == None and end_yr == None): 
    csv_out_path = output_path + f'/allproducts_pCO2_Taylor_stats_Full_version{current_year}{current_month}{current_day}.csv'
else:
    csv_out_path = output_path + f'/allproducts_pCO2_Taylor_stats_version{current_year}{current_month}{current_day}_{start_yr}-{end_yr}.csv'


with fs.open(csv_out_path, 'w') as f:
    df.to_csv(f, index=False)

# Check if saved sucessfully
if fs.exists(csv_out_path):
    print(f"Successfully saved to {csv_out_path}")
else:
    print(f"Failed to save to {csv_out_path}")

In [ ]:
fs.ls(output_path)